# Homework 2

## Reading
* Descriptor: https://realpython.com/python-descriptors/#how-attributes-are-accessed-with-the-lookup-chain}
* Data model: https://docs.python.org/3/reference/datamodel.html (contents related to what we taught)
* Python MRO: https://www.python.org/download/releases/2.3/mro/

## Merge sorted stream
Write a generator function that takes two sorted streams (generators), and return a generator that can produce a merged stream in sorted order.

Bonus point: can you make it generic such that it can merge any number of streams?

In [158]:
def merge_sorted_stream_2(stream_a, stream_b):
    """
    This function is used for merge two sorted streams.
    Both streams are generators already.
    Idea: Compare the smallest elements in the two streams.
    """
    # Initialization: Get the value of the smallest elements in both stream_a and stream_b
    try:
        element_a = next(stream_a)
    except StopIteration:
        element_a = None
    try:
        element_b = next(stream_b)
    except StopIteration:
        element_b = None
    
    # Compare the smallest elements and yield the samller one (There are several different scenarios)
    # Scenario One: When stream_a and stream_b are not completely iterated.
    while (element_a != None) and (element_b != None):
        if (element_a >= element_b):
            yield element_b
            # Update element_b (the second smallest elemnent in stream_b)
            try:
                element_b = next(stream_b)
            except StopIteration:
                element_b = None
        else:
            yield element_a
            # Update element_a
            try:
                element_a = next(stream_a)
            except StopIteration:
                element_a = None   
    # Scenario Two: When stream_a is iterated completely.
    if (element_a == None) and (element_b != None):
        yield element_b
        # Yield all the remaining elements in stream_b
        for element in stream_b:
            yield element
    # Scenario Three: When stream_b is iterated completely
    elif (element_a != None) and (element_b == None):
        yield element_a
        for element in stream_a:
            yield element
    # Scenario Four: When both of the streams are iterated completely
    elif (element_a == None) and (element_b == None):
        yield None

        
def merge_sorted_streams(*streams):
    """
    Function that merge two or more sorted streams.
    Idea: Based on merge_sorted_stream_2, firstly merge the first two streams, then merge this result with the third stream...
    """
    if (len(streams) == 2):
        # print("Only two sorted streams")
        for element in merge_sorted_stream_2(*streams[:2]):
            yield element
    else:
        # print(f"There are {len(streams)} sorted streams")
        for element in merge_sorted_streams(merge_sorted_streams(*streams[:2]), *streams[2:]):
            yield element

In [159]:
# Test:
# Two Streams:
print("Two Streams:")
stream_1 = iter(range(0, 10, 2))
stream_2 = iter(range(1, 10, 2))
for x in merge_sorted_stream_2(stream_1, stream_2):
    print(x)

# More streams:
print("More Streams:")
stream_3 = iter(range(0, 10, 2))
stream_4 = iter(range(1, 10, 2))
stream_5 = iter(range(6, 14, 1))
result = merge_sorted_streams(stream_3, stream_4, stream_5)
for x in result:
    print(x)

Two Streams:
0
1
2
3
4
5
6
7
8
9
More Streams:
0
1
2
3
4
5
6
6
7
7
8
8
9
9
10
11
12
13


## Tree traversal

Unlike linear data structures (Array, Linked List, Queues, Stacks, etc) which have only one logical way to traverse them, trees can be traversed in different ways. Following are the generally used ways for traversing trees.

```
      1
    /  \ 
   2    3
  / \
 4   5
```

Depth First Traversals: 
  * (a) Inorder (Left, Root, Right) : 4 2 5 1 3
  * (b) Preorder (Root, Left, Right) : 1 2 4 5 3
  * (c) Postorder (Left, Right, Root) : 4 5 2 3 1

Define a Tree class with a method that can walk through the tree in different orders. Hint: use generator will make your life a lot easier.

In [160]:
class TreeNode:
    # Initialization: value, right, left
    def __init__(self, value = 0, left = None, right = None):
        self.value = value
        self.left = left
        self.right = right
    
    # In-oder (Left, Root, Right)
    def in_order(self):
        if self.left:
            for nodes in self.left.in_order():
                yield nodes
        yield str(self.value)
        if self.right:
            for nodes in self.right.in_order():
                yield nodes
            
    # Pre-order (Root, Left, Right)
    def pre_order(self):
        yield str(self.value)
        if self.left:
            for nodes in self.left.pre_order():
                yield nodes
        if self.right:
            for nodes in self.right.pre_order():
                yield nodes

    # Post-order (Left, Right, Root)
    def post_order(self):
        if self.left:
            for nodes in self.left.post_order():
                yield nodes
        if self.right:
            for nodes in self.right.post_order():
                yield nodes
        yield str(self.value)

In [161]:
# Test
# Construct the Tree
root = TreeNode(1)
root.left = TreeNode(2)
root.right = TreeNode(3)
root.left.left = TreeNode(4)
root.left.right = TreeNode(5)
# Traverse the Tree
# In-order:
print(' -> '.join(item for item in root.in_order()))
# Pre-order:
print(' -> '.join(item for item in root.pre_order()))
# Post-order:
print(' -> '.join(item for item in root.post_order()))

4 -> 2 -> 5 -> 1 -> 3
1 -> 2 -> 4 -> 5 -> 3
4 -> 5 -> 2 -> 3 -> 1


## Implement a timer
Implement a timer that can print the execution time of your code. Try to implement it both as a decorator and as a context manager to compare the implementations. Can you implement it using one single class? 


In [162]:
import time

class timer:
    """
    Calculate the execution time of my code
    """  
    # __call__ method used for class decorator
    def __call__(self, func):
        def inner(*args, **kwags):
            start_time = time.time()
            result = func(*args, **kwags)
            print(f"Total execution time: {time.time() - start_time} seconds")
            return result
        return inner
    
    def __enter__(self):
        self.start_time = time.time()
        
    def __exit__(self, exc_type, exc_value, traceback):
        print(f"Total execution time: {time.time() - self.start_time} seconds")
        return True

    
# Test
@timer()
def sleep(secs):
    time.sleep(secs)
sleep(10)

with timer() as timer:
    time.sleep(10)

Total execution time: 10.000319242477417 seconds
Total execution time: 10.000093698501587 seconds
